In [239]:
# Part 1
Y = 0
X = 1


def parse(input_str):
    return [list(line) for line in input_str.splitlines()]


def n(yx):
    if yx[Y] == 0:
        return None

    y = yx[Y] - 1
    x = yx[X]
    if grid[y][x] == "#":
        return None

    return (y, x)


def e(yx):
    if yx[X] == WIDTH - 1:
        return None

    y = yx[Y]
    x = yx[X] + 1
    if grid[y][x] == "#":
        return None

    return (y, x)


def s(yx):
    if yx[Y] == HEIGHT - 1:
        return None

    y = yx[Y] + 1
    x = yx[X]
    if grid[y][x] == "#":
        return None

    return (y, x)


def w(yx):
    if yx[X] == 0:
        return None

    y = yx[Y]
    x = yx[X] - 1
    if grid[y][x] == "#":
        return None

    return (y, x)


def neighbours(cell):
    candidates = [n(cell), e(cell), s(cell), w(cell)]
    return [c for c in candidates if c]


def find_start():
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if grid[y][x] == "S":
                return (y, x)

    raise Exception("No start found")


def dump_grid(grid, visited=set()):
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            if (y, x) in visited:
                print("O", end="")
            else:
                print(grid[y][x], end="")
        print("")


# Go
grid = parse(input_str)
HEIGHT = len(grid)
WIDTH = len(grid[0])

steps = 64

active_frontier = set([find_start()])

for i in range(steps):
    next_frontier = set()
    for cell in active_frontier:
        for next in neighbours(cell):
            next_frontier.add(next)

    active_frontier = next_frontier
    # print(f"After {i + 1} steps, visited {len(frontier)} cells")

print("Part 1:", len(active_frontier))

Part 1: 42


In [250]:
# Part 2
import timeit

def get(y, x):
    return grid[y % HEIGHT][x % WIDTH]

def neighbours_infinite(yx):
    n = (yx[Y] - 1, yx[X])
    e = (yx[Y], yx[X] + 1)
    s = (yx[Y] + 1, yx[X])
    w = (yx[Y], yx[X] - 1)

    return [c for c in [n, e, s, w] if get(c[Y], c[X]) != "#"]

def neighbours_infinite_mod(yx):
    n = ((yx[Y] - 1) % HEIGHT, yx[X])
    e = (yx[Y], (yx[X] + 1) % WIDTH)
    s = ((yx[Y] + 1) % HEIGHT, yx[X])
    w = (yx[Y], (yx[X] - 1) % WIDTH)

    return [c for c in [n, e, s, w] if get(c[Y], c[X]) != "#"]
    
def n2(grid_yx, yx):
    y = (yx[Y] - 1) % HEIGHT
    x = yx[X]
    if grid[y][x] == "#":
        return None
    
    # did we move onto another grid?
    if y == HEIGHT - 1:
        grid_yx = inc_yx(grid_yx, Y, -1)

    return (grid_yx, (y, x))

def s2(grid_yx, yx):
    y = (yx[Y] + 1) % HEIGHT
    x = yx[X]
    if grid[y][x] == "#":
        return None
    
    # did we move onto another grid?
    if y == 0:
        grid_yx = inc_yx(grid_yx, Y, 1)

    return (grid_yx, (y, x))

def w2(grid_yx, yx):
    y = yx[Y]
    x = (yx[X] - 1) % WIDTH
    if grid[y][x] == "#":
        return None
    
    # did we move onto another grid?
    if x == WIDTH - 1:
        grid_yx = inc_yx(grid_yx, X, -1)

    return (grid_yx, (y, x))

def e2(grid_yx, yx):
    y = yx[Y]
    x = (yx[X] + 1) % WIDTH
    if grid[y][x] == "#":
        return None
    
    # did we move onto another grid?
    if x == 0:
        grid_yx = inc_yx(grid_yx, X, 1)

    return (grid_yx, (y, x))

def inc_yx(yx, field, delta):
    if field == Y:
       return (yx[Y] + delta, yx[X])
    else:
       return (yx[Y], yx[X] + delta)
    
def add_yx(yx1, yx2):
    if (yx2 == (0, 0)):
        return yx1
    else:
        return (yx1[Y] + yx2[Y], yx1[X] + yx2[X])
    

def neighbours_infinite_4(grid_yx, yx, start_yx):
    candidates = []

    # print(grid_yx, yx, start_yx)

    # Only expore n if we're north of the origin
    if grid_yx[Y] < 0 or yx[Y] <= start_yx[Y]:
        candidates.append(n2(grid_yx, yx))
    elif grid_yx[Y] > 0 or yx[Y] >= start_yx[Y]:
        candidates.append(s2(grid_yx, yx))

    if grid_yx[X] < 0 or yx[X] <= start_yx[X]:
        candidates.append(w2(grid_yx, yx)) 
    elif grid_yx[X] > 0 or yx[X] >= start_yx[X]:
        candidates.append(e2(grid_yx, yx))

    return [c for c in candidates if c]

def neighbours_infinite_3(grid_yx, yx):
    candidates = [n2(grid_yx, yx), e2(grid_yx, yx), s2(grid_yx, yx), w2(grid_yx, yx)]

    return [c for c in candidates if c]

# Precompute neighbours for each cell
cache = {}
for y in range(HEIGHT):
    for x in range(WIDTH):
        cell = (y, x)
        cache[cell] = neighbours_infinite_3((0, 0), cell)
        # print(cell, "->", cache[cell])

def dump_grid2(grid, visited=set()):
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            if (y, x) in visited:
                print("O", end="")
            else:
                print(grid[y][x], end="")
        print("")

# Go
steps = 100

# 500 -> 50s
# 1000 -> bailed after 5m

frontier_a = set()
frontier_b = set()

active_frontier = frontier_a
other_frontier = frontier_b

start_yx = find_start()

added_last_step = set([((0,0), start_yx)])



# Intuition: we only need to iterate over the cells which are _new_ to the frontier.
# Most of the cells we have already visited. We can handle those by somehow deferring
# to the i-2th frontier.

for i in range(steps):
    count_n = 0
    count_prev = 0
    count_iter = 0

    last = added_last_step
    added_last_step = set()

    start_step = timeit.default_timer()

    for (g, cell) in last:
        count_iter += 1

        # cc = cache[cell]
        # print(cc)

        # neighbours = neighbours_infinite_4(g, cell, start_yx)
        neighbours = cache[cell]
        count_n += len(neighbours)
        # print(count_n)
       
        

        for (grid_delta, cell_next) in neighbours:
            grid_next = add_yx(g, grid_delta)
            

            if (grid_next, cell_next) in other_frontier:
                count_prev += 1
            else:
                added_last_step.add((grid_next, cell_next))
                other_frontier.add((grid_next, cell_next))

    # other_frontier.update(added_last_step)

    step_time = timeit.default_timer() - start_step

    print(f"Step {i}: {count_iter} iterations / {count_n} neighbours / {count_n - count_prev} new")
    print(f" -> {'{:.3e}'.format(step_time)}")
    # print(f" -> cheeky intersect {len(added_last_step.intersection(active_frontier))}")

    # cycle between two frontier sets
    tmp = other_frontier
    other_frontier = active_frontier
    active_frontier = tmp

    dump_grid(grid, [c for (g, c) in active_frontier if g == (0, 0)])

    
    # print(f"After {i + 1} steps, visited {len(frontier)} cells")

# print(frontier)

print("Part 2:", len(active_frontier))



Step 0: 1 iterations / 2 neighbours / 2 new
 -> 6.750e-06
...........
.....###.#.
.###.##..#.
..#.#...#..
....#O#....
.##.OS####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
Step 1: 2 iterations / 5 neighbours / 4 new
 -> 5.167e-06
...........
.....###.#.
.###.##..#.
..#.#O..#..
....#.#....
.##O.O####.
.##.O#...#.
.......##..
.##.#.####.
.##..##.##.
...........
Step 2: 4 iterations / 10 neighbours / 4 new
 -> 6.375e-06
...........
.....###.#.
.###.##..#.
..#.#.O.#..
...O#O#....
.##.OS####.
.##O.#...#.
....O..##..
.##.#.####.
.##..##.##.
...........
Step 3: 4 iterations / 11 neighbours / 5 new
 -> 7.791e-06
...........
.....###.#.
.###.##..#.
..#O#O.O#..
..O.#.#....
.##O.O####.
.##.O#...#.
...O.O.##..
.##.#.####.
.##..##.##.
...........
Step 4: 5 iterations / 13 neighbours / 7 new
 -> 9.750e-06
...........
.....###.#.
.###.##O.#.
..#.#.O.#..
.O.O#O#O...
.##.OS####.
.##O.#...#.
..O.O.O##..
.##O#O####.
.##..##.##.
...........
Step 5: 7 iterations / 14 neighbours / 7 new
 -

In [256]:
start = timeit.default_timer()

c = 0
for i in range(26501365):
    c += 1

end = timeit.default_timer()
print(end - start)

1.3768115409984603


In [255]:
import numpy

start = timeit.default_timer()

c = 0
for i in numpy.arange(26501365):
    c += 1

end = timeit.default_timer()
print(end - start)

3.1551041249986156


In [261]:
cells = len(input_str.splitlines()) * len(input_str.splitlines()[0])

2 ** cells

9457047918193045552203834799927684696698937494298070671166002533166648330751576426870145127798713481269762459897283601429351802189289021308469456445823333131952001022447806140226981289253169494076318484762691410208079277983511733506376671464104612475109379775003226423649546531478192811735315222514057517973256138320238158068429444849119714124052369072780080332485492924140078115253255290217692630595542792853160551071086425505696352677710142833609156827737114105436547105217815219162360319493542274397104607133817675326809428466544422428169376213092309128919284848153763767582433042025963132860509684508992119008949102159015576957230303352762604167553024807065696528431109968921355153976857564556491131092928383335265375471224365963022981662961070342238065529013170390433604532618261119640341546271512404078353939096583837956059327471874262075362326983163293690419224367272930643779631911558987680635096959487811873900873690129334614916460834534521152604624541618973550722576423424623445513665969639

In [238]:
# Test Input
input_str="""
...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
""".strip()


In [257]:
# Input
input_str="""
...................................................................................................................................
.#...............#..#...........#....#................................................##......##..........#..............#.........
......#...................#...#..#..........#.#.......#.....................#.............##.....#....#......#..#........#.......#.
.#....#...#...#..#.................#.....#....#.#..#..........................#.....#...........#.............#....................
.#.............................##...##......#........#......................#.........##...#..#.................##.........#.#.....
....#.#....#.................#..#..#........#...#.#.#..#......................#........#....#..............#.....##................
....#.....#......#....#.##....#.##........#.....#......#........................#..........#.............#...................#.....
...........#...........##.#..........##.....#...#...##........#.................#.......#.............#......#..#..........#.......
..#...........#.............#...................................#.....................##...........#..............#.#....#.........
....#.....#.........#....#......##..............#...#........#.##....#........#..#....#..........#...................#..........#..
............#.........#............#.#...........#.........#.........................#......#..#......#..#..#.......#.....#........
.........#.................#........#..#.....#.............#........................#..#...#...#..#..#.....#...#..#......#....#....
..........#.#.#...............#.........#....##.#..........##......##...............#...#...###.##...#.......#..............#......
....#.........#...#........#.##.............#............#........#....##......................#..........#......#...#.......#.....
.....##..#......#...#........#................#............................#..........#............##..#...#.....#..#........##....
..#........#.#.###.....#....#..#.....#.....#..........................#.....#...........#.##..........#..........#...#...#.........
...................#.................................#.....#....#...#...#...................#..#......#....#.#.##................#.
.........#...#..#.#....#........#..........#................#.#....#...#.............................#..................#..#.#.....
...........#...#.#...................#..........................#..........#...................#...............#.###........#..#...
......#.......#...............#.........#.#...........#.####............#...#............#...##..##..........#............#........
..............................#..#.###..................#.....#......#..#.#..#...#..............#...#.....#.##..#.............#....
...............#.......#.#....#......#.#..........#..#.................#.#........#.......#.................#...............#...##.
............#.................#....................#............#....#.....#...............#........###........#.#..#.##...#.......
...#.....#.#..#.....#....#...#......#...........................#..#..#.......................#..............#..........#..........
.#....##..............#.##....#...#................#............#..........#......##....................##....#.#..#......#..##....
.....#.........#...##.#.#......................#....#...#...#.............#...................#...#..#...#...###......#...#....#...
........##......#.....#......................................#....###.#...#..........#.............#.....#.....#..........#.#......
........##.#...##.........##.....................#.#...........#........#.......................#...#.....#..............#.#.......
..........#.##.#..#..#..#.......#.....................................#.......##....#................#....#.....#...##.#....#..##..
..##.......#.........#..................##........#.....#.....................#.....#...................#.....#.#.........#........
....#........#.....#...................#.#..#..........................#...........#.............................#......#....#.....
..........................................#............#..#.......................#...#.#..#..................#.#............##....
...#..#....#......#....................................................................#..............#.#................#.......#.
..#.....##.............#.#.#...........#........#.................#..................#....................#..............#..###....
...##..#............#...#..#.........#..#.....#...........................#.....#....#...#...............#....#.............#...#..
...........#.......#...............................#..#.#...#.#...#...#........#.............##......................#...........#.
...............#..................#...............#.............#..#......................#......................##..........#.....
...........#..#....#.##...........#....#.##.#..........#...#.........#..........#.#..............................##.........#..#.#.
......#...#.#..##..................##................#####.#........#....#..#........#....#.....#................##..........#.....
....#.#........#.....#.......................##.......................#..#.#...........#.........................#..#..............
...#...........#..............#.......#.#....#......#...#....#.....#.....#...............#...........................#......#....#.
....#...........#..#.................#...##....#.............#..................#...........#.#.#............................#.....
........#..................#..#.#.........................###...#..#.#.#..##......#........##..#.....#..........#.....#.........#..
.....#.#.#....#...#..............#.#..#.......#.........###..#..........#.......#..#....................#....................##....
........#.......#...............##................#..............................#...#.#.....#....#.....##........#.#......#.......
.....#.##.....................#....#......###..#...........#........#....#..........#...#....#.........................#.......#...
..#.....#...#................#...##.#.......#....#.....#...............#.#....................#...#.......#..........#.............
........#.............#.#................#..#.....#....#.#......................#.....#.................##..#......................
..#..........#..........#..............#.........#..#.........#..............#.#.....#......###...#.....................#..........
......................#...#.#.#..#........#....###......##.........#.............#........#..#...#........#....................#...
..#..#...................#...#..#..#....#..##............#...#.............#.....#....##...............#...................##..#...
...#................###...............#...........#..............................#...#.........#.....#..#..#.................#...#.
....#.#..................##....#...#...#.........#....#.......#......#..........##..#..#..##.........#.....................#.......
......#..........#....#.....#.#.....#...#..........#.#......##.#......#............#.....#.....###.......#.#.#..#..................
...#.#.........##........#...#.....#......#...#.......#..................................#.........................................
..............#........................#.......###..#.......#.......#.#.......#....#.............#.......#...#.....#...............
.#..#..........#...........##..............#...#.........#.....##.#....................................#.......................#...
...................#........#....#.#...........#............#.#.#.##...##............#.#.....................#.....................
...........#....#........#...........#...#..##....#.#........#..#......#..##....................#...#..............................
.............#..........#.....#.......##......#......##.....#......##............................##.........#.........#.#..........
..............#.#...#.....#.....##..............#.......#.#..#.#......#...................#....#........#..........................
.....................##..#.#.....#..........#......#......#..#....#.#.............#...#......#..#..................................
...............#.#.##........#.........#..........#.....#............#...#.......##.........#........##....#..#....................
...........##....#.............#.................#...#..#...#........#...#.#..#..#.................#...#.....#.....................
..................................#........#..........#...........##.#.##....#.#..#............#.#.............#...................
.................................................................S.................................................................
......#..................###.##........#..#.....#....................#............#....#.#...#.#....#.###......#....##.......#.....
......................#.##......#.#......#..#....#.....................#...........#.....#..............#.#..#......#..............
..............................#..#............#......##.#.........#........#........#........................#.....................
..........#........#..........##.#.#..#...................#.#.#............#...............#...#.......#....#...##.................
............#.....#..........#.#.......#.#.........#....#.#.....#....#..........#.......#..............#........#..#....##.........
............#.............................#.................#......#..................#..#.....#..#...............#...##...........
.................###....#...#.#..............#.#..............#.......#...##....#.....#............#...#..##.##........#.......#.#.
....................#..#..##.#.#..##.....##...............#........#......##................#......................................
...................#.......#....#....#.......#..#.............#..........#.#..........#.##.............#...........................
....................................#..#...#....#......##..#.......#...#................#.....#....#.#.......#...#.#...............
.##...##.......#........#...#...#......#...#..............#..............##..##...#.........#...##...#.....................#..#....
.#....#.............#.#....#....#...............#..#....#..........#.......#......#.....#.................#.....................#..
.#............................#......##......#..#.#........#.....................#.#..#..##..#......#....................#..#......
.......#............#....#....#...#..#........#...............#......#.......#..................#......#...#.............#.........
..#.................#.####..#...........#...........#..............#..#.##..##....#................#.....#.................#...#...
.....#..............#...#........#................#....##.##..#..............#..##......#..#.........#................##...#...#...
.........##..........#.......#..........#.#....#......#...........###...........#.......................#..#.#................#.#..
.......#......#.......#......................................#..............#.................#..#........#..........#..........#..
.......#...#..............#.......##...........#.#.#......#.........#.......#....#........#..#..#........#.................#.......
.#.....##.#.#.................#.###...................##......#....##..##................#...............#.........................
..........##..#............#.#............#.......#..#.............#.......##.....##..#...##...#...................#...##.#...#....
.##...#.....##.............#.........#...............#.............................#...........#....................#.....##.......
..........##.#.............#............#...#..#....#...........#.....#..........#...#.......#..#.#................................
..#.....................................#.....................#......#...............#...#........#......................#.##......
.........#.#....#....#...............#......#...#........##...........#.......#....#....#...#......#.............#.....#....##.....
.............#....................#...........#...................#.#...................#.#..#....................#..........#.....
.....#.............#..#...................#................#......#...#.......#......#....#..##...................#.#.##.....#..#..
.#..##...#........#......................#...##.#..##.........................#............#.#............#..#.....................
.#...................#................#..............##...........#........#.#.....#.#...#.................#...##...#....#..#......
....#.......#..#...#.#...#.............#..#.##...#....#....#.......................#......#....#........#..#.......................
........###.......##..................#.......#....#.##....#..................##..........................#..#........#.........#..
.##.......##.....#.........#..........#......#...............#........#.#..#..........#...............#.#...#..........#...........
.#...##............#.........#..........##.....#..#......#......#....##...#...##.....#..#..............#.#.......##....#.#.#...#.#.
...........#...........................#..........#.#..........#....#..........#....................#.....##.............#.......#.
.........#..................##............#..................#................#.#.......#.............###..#....#.......#........#.
.......#...............#................#........#.....#.................##......#.......................#...............#.........
...##............#......#....................#...##.............#......#....#.....##.........................#.............#.......
.#.......#.#.#..#.......#......#..............#...#.........#......##.##.....#.....................#.....#.....#......##......#....
.#...................#.#....#.....#..........#..#...#.......................#...#....#.....................#....#...###.....#......
......#.....#......#....#.#.................#.........#.##...............##...................#..#......#..#.............#.........
..#........#....#.#............................................#......#...#.......#.....................#...................#......
.........#...#.#.....................................#....#..........................................##...#........................
..#...............##.#............#....#........##.#....#..................#...#...........#...#..#..#....#..........#....#....#...
............#.#.......###.......##.##.....................#...#.#...#..#....#...##............#.#.#..............#..#.#...#........
................................#..#.#....................#..#.#......#..###...............#..#...#...##.#.......#.......#.........
....#........#.......#..#..........##.........................#....#......##..#................#......#..#......#.#......#.#.......
.#..............#.#....#..........#.#....................#......#............................#....#......##...#.#.......#..........
....#.....##...#........#..#...........#...#.........#...............#.................#.#..#.................#..............#.....
...........................................##.............#.#.....#......#................#...#........#.......#..#.#..............
......#.....#..#.......##....#..........#......................#............#..........#...#...............#..#..#....#...#...#....
..........#...##.....................#.................##...............##.............#..##...........#.#............#.....##.....
.#.#........................#...#.....#...........................#........................#........#.....#.#.#...........#........
..#...#....................#..#...#.........#...#........#.#...#...................#..#.#.....................#...........#........
...#.#.......#...#.................#......#...#...#....................##.................#..........##.......#..#.....#........#..
..................#.......#................#...............#.#....#....#......................#.....#.......#.........##...#.......
..#.......#.#..............#..###........##.#.................................##..........#........#..........#.....#.....#........
.........##.....#.........#..........#...........#.#.................#.........#..............#...............#......#.#...##......
...............#.......#.#...#...........#.....#...#........................#..............#.......#.#......#....#......#....#.....
.................##.#.#........##..........................................#....#.........##......#.......#..#..................#..
...#..##......#.....#.............##.#..##....#...............................##......###.#....###.#.........#...........#...#.##..
..#..#..............#.....#.....#..#..##.......#..##.....#.................#...#................#..#........#...##...#.............
.#..................#............#.#..................................................#......#.#.......###...........#........##.#.
............#..........#....#.....#...#.....#........#.#.........................#.#....#....#.##..........#......#................
....#....#..#....................#.....#........##......#...#.........##.....#...##...#...##.##..............#....#......#..#...#..
...................................................................................................................................
""".strip()